# General Imports

In [7]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [8]:
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import seaborn as sns

In [9]:
from icecream import ic

In [10]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torchvision import transforms as transform_lib
from pytorch_lightning.callbacks import TQDMProgressBar
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [11]:
from dataset.cityscapes import Cityscapes
from dataset.cityscapes import CityscapesDataModule
from dataset.woodscape import WoodScapeDataset
from dataset.woodscape import WoodScapeDataModule
from dataset import WoodScapeSoilingDataset
from dataset import WoodScapeSoilingDataModule

In [12]:
from utils.display_images import denormalize_img
from utils import show_dataset_image, show_dataset_mask
from utils import show_prediction_images, show_prediction_uncertainty_images

In [13]:
from deeplab_v3p import DeepLabV3PlusModule
from dropblock import DropBlock2D

In [14]:
from ls_ood_detect.uncertainty_estimation import Hook
from ls_ood_detect.uncertainty_estimation import deeplabv3p_get_ls_mcd_samples
from ls_ood_detect.uncertainty_estimation import get_dl_h_z
from ls_ood_detect.ood_detection_dataset import build_ood_detection_ds
from ls_ood_detect.dimensionality_reduction import plot_samples_pacmap
from ls_ood_detect.detectors import KDEClassifier
from ls_ood_detect.metrics import get_ood_detector_results, plot_roc_ood_detector

# OoD/Anomaly Detection Steps

Steps for using the package:

1. Load you Dataloader Pytorch-Lightning Module
2. Load your trained DNN PyTorch-Lightning Module
3. Add Hook to DNN Module for MC samples extraction
4. Get Monte-Carlo (MC) samples for In-Distribution (InD) samples dataloader, and Out-of-Distribution (OoD) samples dataloader
5. Get Entropy from InD and OoD MC samples
6. Build OoD Detection dataset (with InD and OoD samples)
7. Build OoD data-driven Detector (classifier)
8. Show OoD performance results

# Datasets

## Woodscape

### Datamodules

In [9]:
ws_dataset_path = './Data/DATASETS/WoodScape/'
batch_size = 1

cmap = {0: [0, 0, 0],  # "void"
        1: [128, 64, 128],  # "road",
        2: [69, 76, 11],  # "lanemarks",
        3: [0, 255, 0],  # "curb",
        4: [220, 20, 60],  # "person",
        5: [255, 0, 0],  # "rider",
        6: [0, 0, 142],  # "vehicles",
        7: [119, 11, 32],  # "bicycle",
        8: [0, 0, 230],  # "motorcycle",
        9: [220, 220, 0]  # "traffic_sign",
        }

# same values as in VainF Repository! - Probably not the best Values for Woodscapes!
ws_dlv3p_norm_mean = [0.485, 0.456, 0.406]
ws_dlv3p_norm_std = [0.229, 0.224, 0.225]

__Woodscape img_size (483, 640)__

In [10]:
ws_dm_normal_dlv3p = WoodScapeDataModule(dataset_dir=ws_dataset_path,
                                         img_size=(483, 640),
                                         batch_size=batch_size,
                                         default_transforms=True,
                                         label_colours=cmap,
                                         norm_mean=ws_dlv3p_norm_mean,
                                         norm_std=ws_dlv3p_norm_std,
                                         seed=9290,
                                         drop_last=True)
ws_dm_normal_dlv3p.setup()

__Woodscape img_size (256, 512)__

In [11]:
ws_dm_normal_dlv3p_256_512 = WoodScapeDataModule(dataset_dir=ws_dataset_path,
                                         img_size=(256, 512),
                                         batch_size=batch_size,
                                         default_transforms=True,
                                         label_colours=cmap,
                                         norm_mean=ws_dlv3p_norm_mean,
                                         norm_std=ws_dlv3p_norm_std,
                                         seed=9290,
                                         drop_last=True)
ws_dm_normal_dlv3p_256_512.setup()

__Woodscape-Anomalies img_size (483, 640)__

In [12]:
ws_dm_anomal_dlv3p = WoodScapeDataModule(dataset_dir=ws_dataset_path,
                                         img_size=(483, 640),
                                         batch_size=batch_size,
                                         default_transforms=True,
                                         label_colours=cmap,
                                         norm_mean=ws_dlv3p_norm_mean,
                                         norm_std=ws_dlv3p_norm_std,
                                         seed=9290,
                                         drop_last=True)
ws_dm_anomal_dlv3p.setup()

### Woodscape dataloaders

In [13]:
ws_dlv3p_train_loader = ws_dm_normal_dlv3p.train_dataloader()
ws_dlv3p_valid_loader = ws_dm_normal_dlv3p.val_dataloader()
ws_dlv3p_test_loader = ws_dm_normal_dlv3p.test_dataloader()

In [14]:
ws_dlv3p_anomaly_valid_loader = ws_dm_anomal_dlv3p.anomaly_val_dataloader()
ws_dlv3p_anomaly_test_loader = ws_dm_anomal_dlv3p.anomaly_test_dataloader()

In [15]:
ws_256512_dlv3p_valid_loader = ws_dm_normal_dlv3p_256_512.val_dataloader()
ws_256512_dlv3p_test_loader = ws_dm_normal_dlv3p_256_512.test_dataloader()

In [16]:
ic(len(ws_dlv3p_train_loader));
ic(len(ws_dlv3p_valid_loader));
ic(len(ws_dlv3p_test_loader));
ic(len(ws_dlv3p_anomaly_valid_loader));
ic(len(ws_dlv3p_anomaly_test_loader));
ic(len(ws_256512_dlv3p_valid_loader));
ic(len(ws_256512_dlv3p_test_loader));

## Cityscapes

### Datamodules

In [17]:
dataset_path = './Data/DATASETS/CityScapes'
batch_size = 1

__Cityscapes img_size (256, 512)__

In [18]:
cs_dm_normal_dlv3p = CityscapesDataModule(data_dir=dataset_path,
                                          batch_size=batch_size,
                                          target_type='semantic',
                                          img_size=(256, 512),
                                          num_workers=10,
                                          drop_last=True,
                                          default_transforms=True, # Here this should be True!
                                          default_img_mask_transforms=False) # And here this should be False!


__Cityscapes img_size (483, 640)__

In [19]:

cs_483640_dm_dlv3p = CityscapesDataModule(data_dir=dataset_path,
                                          batch_size=batch_size,
                                          target_type='semantic',
                                          img_size=(483, 640),
                                          num_workers=10,
                                          drop_last=True,
                                          default_transforms=True, # Here this should be True!
                                          default_img_mask_transforms=False) # And here this should be False!        

__Cityscapes-Anomalies img_size (256, 512)__

In [20]:
cs_dm_anomal_dlv3p = CityscapesDataModule(data_dir=dataset_path,
                                          batch_size=batch_size,
                                          target_type='semantic',
                                          img_size=(256, 512),
                                          num_workers=10,
                                          drop_last=True,
                                          default_transforms=False, # Here this should be False!
                                          default_img_mask_transforms=True) # And here this should be True! (Enable Anomalies)

### Cityscapes Dataloaders

In [21]:
cs_dlv3p_train_loader = cs_dm_normal_dlv3p.train_dataloader()
cs_dlv3p_valid_loader = cs_dm_normal_dlv3p.val_dataloader()
cs_dlv3p_test_loader = cs_dm_normal_dlv3p.test_dataloader()

In [22]:
cs_dlv3p_anomaly_valid_loader = cs_dm_anomal_dlv3p.anomaly_val_dataloader()
cs_dlv3p_anomaly_test_loader = cs_dm_anomal_dlv3p.anomaly_test_dataloader()

In [23]:
cs_483640_dlv3p_valid_loader = cs_483640_dm_dlv3p.val_dataloader()
cs_483640_dlv3p_test_loader = cs_483640_dm_dlv3p.test_dataloader()

In [24]:
ic(len(cs_dlv3p_train_loader));
ic(len(cs_dlv3p_valid_loader));
ic(len(cs_dlv3p_test_loader));
ic(len(cs_dlv3p_anomaly_valid_loader));
ic(len(cs_dlv3p_anomaly_test_loader));
ic(len(cs_483640_dlv3p_valid_loader));
ic(len(cs_483640_dlv3p_test_loader));

## Woodscape-Soiling

### Datamodules

__Woodscape Soling OoD DeeplabV3+ (483, 640)__

In [25]:
woodscape_soil_483640_dm = WoodScapeSoilingDataModule(dataset_dir="./Data/DATASETS/WoodScape/soiling_dataset/",
                                                      img_size=(483, 640),
                                                      batch_size=1,
                                                      default_transforms=True,
                                                      seed=9290)
woodscape_soil_483640_dm.setup()

__Woodscape Soling OoD DeeplabV3+ (256, 512)__

In [26]:
woodscape_soil_256512_dm = WoodScapeSoilingDataModule(dataset_dir="./Data/DATASETS/WoodScape/soiling_dataset/",
                                                      img_size=(256, 512),
                                                      batch_size=1,
                                                      default_transforms=True,
                                                      seed=9290)
woodscape_soil_256512_dm.setup()

### Dataloaders

In [27]:
ws_soiling_483640_valid_loader = woodscape_soil_483640_dm.val_dataloader()
ws_soiling_483640_test_loader = woodscape_soil_483640_dm.test_dataloader()

In [28]:
ws_soiling_256512_valid_loader = woodscape_soil_256512_dm.val_dataloader()
ws_soiling_256512_test_loader = woodscape_soil_256512_dm.test_dataloader()

In [33]:
ic(len(ws_soiling_483640_valid_loader));
ic(len(ws_soiling_483640_test_loader));
ic(len(ws_soiling_256512_valid_loader));
ic(len(ws_soiling_256512_test_loader));

# Deeplabv3+ Woodscape Trained Model

In [15]:
ws_dlv3p_path = "./lightning_logs/version_66126/checkpoints/last.ckpt"
ws_dlv3p_model = DeepLabV3PlusModule.load_from_checkpoint(checkpoint_path=ws_dlv3p_path)


In [16]:
ic(ws_dlv3p_model.pred_loss_type);
ic(ws_dlv3p_model.n_class);

In [36]:
ws_dlv3p_model.eval();

# Add Hook Deeplabv3+ Woodscape

In [18]:
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1);
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1.block_size);
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1.drop_prob);
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1.training);

In [38]:
ws_dlv3p_hook_dropblock2d_layer = Hook(ws_dlv3p_model.deeplab_v3plus_model.drop_block1)

# Get Monte Carlo Samples

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [40]:
# Monte Carlo Dropout - Enable Dropout @ Test Time!
def deeplabv3p_apply_dropout(m):
    if type(m) == torch.nn.Dropout or type(m) == DropBlock2D:
        m.train()

In [41]:
mc_samples = 16

In [42]:
ws_dlv3p_model.deeplab_v3plus_model.to(device);
ws_dlv3p_model.deeplab_v3plus_model.eval(); 
ws_dlv3p_model.deeplab_v3plus_model.apply(deeplabv3p_apply_dropout); # enable dropout

In [43]:
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1.drop_prob);
ic(ws_dlv3p_model.deeplab_v3plus_model.drop_block1.training);

In [ ]:
ws_dlv3p_ws_normal_train_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                      ws_dlv3p_train_loader,
                                                                      mc_samples,
                                                                      ws_dlv3p_hook_dropblock2d_layer)

In [50]:
ws_dlv3p_ws_normal_valid_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                      ws_dlv3p_valid_loader,
                                                                      mc_samples,
                                                                      ws_dlv3p_hook_dropblock2d_layer)

ws_dlv3p_ws_normal_test_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                     ws_dlv3p_test_loader,
                                                                     mc_samples,
                                                                     ws_dlv3p_hook_dropblock2d_layer)

In [44]:
ws_dlv3p_ws_anomal_valid_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                      ws_dlv3p_anomaly_valid_loader,
                                                                      mc_samples,
                                                                      ws_dlv3p_hook_dropblock2d_layer)

ws_dlv3p_ws_anomal_test_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                     ws_dlv3p_anomaly_test_loader,
                                                                     mc_samples,
                                                                     ws_dlv3p_hook_dropblock2d_layer)

In [45]:
ws_dlv3p_cs_483640_valid_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                      cs_483640_dlv3p_valid_loader,
                                                                      mc_samples,
                                                                      ws_dlv3p_hook_dropblock2d_layer)

ws_dlv3p_cs_483640_test_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                     cs_483640_dlv3p_test_loader,
                                                                     mc_samples,
                                                                     ws_dlv3p_hook_dropblock2d_layer)

In [46]:
ws_dlv3p_ws_soiling_483640_valid_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                              ws_soiling_483640_valid_loader,
                                                                              mc_samples,
                                                                              ws_dlv3p_hook_dropblock2d_layer)

ws_dlv3p_ws_soiling_483640_test_16mc_samples = deeplabv3p_get_ls_mcd_samples(ws_dlv3p_model,
                                                                             ws_soiling_483640_test_loader,
                                                                             mc_samples,
                                                                             ws_dlv3p_hook_dropblock2d_layer)

In [58]:
torch.save(ws_dlv3p_ws_normal_train_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_train_16mc_samples.pt')


In [52]:
torch.save(ws_dlv3p_ws_normal_valid_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_valid_16mc_samples.pt')
torch.save(ws_dlv3p_ws_normal_test_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_test_16mc_samples.pt')

In [47]:
torch.save(ws_dlv3p_ws_anomal_valid_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_anomal_valid_16mc_samples.pt')
torch.save(ws_dlv3p_ws_anomal_test_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_anomal_test_16mc_samples.pt')

In [49]:
torch.save(ws_dlv3p_cs_483640_valid_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_cs_483640_valid_16mc_samples.pt')
torch.save(ws_dlv3p_cs_483640_test_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_cs_483640_test_16mc_samples.pt')

In [50]:
torch.save(ws_dlv3p_ws_soiling_483640_valid_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_soiling_483640_valid_16mc_samples.pt')
torch.save(ws_dlv3p_ws_soiling_483640_test_16mc_samples,
           './Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_soiling_483640_test_16mc_samples.pt')

In [53]:
ws_dlv3p_ws_normal_train_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_train_16mc_samples.pt')
ws_dlv3p_ws_normal_valid_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_valid_16mc_samples.pt')
ws_dlv3p_ws_normal_test_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_normal_test_16mc_samples.pt')

ws_dlv3p_ws_anomal_valid_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_anomal_valid_16mc_samples.pt')
ws_dlv3p_ws_anomal_test_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_anomal_test_16mc_samples.pt')

ws_dlv3p_cs_483640_valid_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_cs_483640_valid_16mc_samples.pt')
ws_dlv3p_cs_483640_test_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_cs_483640_test_16mc_samples.pt')

ws_dlv3p_ws_soiling_483640_valid_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_soiling_483640_valid_16mc_samples.pt')
ws_dlv3p_ws_soiling_483640_test_16mc_samples = torch.load('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/mc_samples/ws_dlv3p_ws_soiling_483640_test_16mc_samples.pt')

# Get Entropy

In [55]:
_, ws_dlv3p_h_z_ws_normal_train_samples_np = get_dl_h_z(ws_dlv3p_ws_normal_train_16mc_samples, mcd_samples_nro=16)

In [57]:
_, ws_dlv3p_h_z_ws_normal_valid_samples_np = get_dl_h_z(ws_dlv3p_ws_normal_valid_16mc_samples, mcd_samples_nro=16)

In [58]:
_, ws_dlv3p_h_z_ws_normal_test_samples_np = get_dl_h_z(ws_dlv3p_ws_normal_test_16mc_samples, mcd_samples_nro=16)

In [59]:
_, ws_dlv3p_h_z_ws_anomal_valid_samples_np = get_dl_h_z(ws_dlv3p_ws_anomal_valid_16mc_samples, mcd_samples_nro=16)

In [60]:
_, ws_dlv3p_h_z_ws_anomal_test_samples_np = get_dl_h_z(ws_dlv3p_ws_anomal_test_16mc_samples, mcd_samples_nro=16)

In [61]:
_, ws_dlv3p_h_z_cs_483640_valid_samples_np = get_dl_h_z(ws_dlv3p_cs_483640_valid_16mc_samples, mcd_samples_nro=16)

In [62]:
_, ws_dlv3p_h_z_cs_483640_test_samples_np = get_dl_h_z(ws_dlv3p_cs_483640_test_16mc_samples, mcd_samples_nro=16)

In [63]:
_, ws_dlv3p_h_z_ws_soil_483640_valid_samples_np = get_dl_h_z(ws_dlv3p_ws_soiling_483640_valid_16mc_samples, mcd_samples_nro=16)

In [64]:
_, ws_dlv3p_h_z_ws_soil_483640_test_samples_np = get_dl_h_z(ws_dlv3p_ws_soiling_483640_test_16mc_samples, mcd_samples_nro=16)

In [56]:
np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_normal_train_samples_np',
        ws_dlv3p_h_z_ws_normal_train_samples_np)

In [12]:
np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_normal_valid_samples_np',
        ws_dlv3p_h_z_ws_normal_valid_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_normal_test_samples_np',
        ws_dlv3p_h_z_ws_normal_test_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_anomal_valid_samples_np',
        ws_dlv3p_h_z_ws_anomal_valid_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_anomal_test_samples_np',
        ws_dlv3p_h_z_ws_anomal_test_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_cs_483640_valid_samples_np',
        ws_dlv3p_h_z_cs_483640_valid_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_cs_483640_test_samples_np',
        ws_dlv3p_h_z_cs_483640_test_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_soil_483640_valid_samples_np',
        ws_dlv3p_h_z_ws_soil_483640_valid_samples_np)

np.save('./Data/EXPERIMENTS/CVPR-2024/semantic_segmentation/deeplabv3p/embeddings/woodscape/entropy/ws_dlv3p_h_z_ws_soil_483640_test_samples_np',
        ws_dlv3p_h_z_ws_soil_483640_test_samples_np)